# Capitulo 9: Prompts Complejos desde Cero

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu clave API y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

# Correccion de ruta para importar pistas
import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Leccion

Felicidades por llegar al ultimo capitulo! Ahora es momento de unir todo y aprender a **crear prompts unicos y complejos**.

A continuacion, utilizaras una **estructura guiada que recomendamos para prompts complejos**. En las partes posteriores de este capitulo, te mostraremos algunos prompts especificos de la industria y explicaremos como esos prompts estan estructurados de manera similar.

**Nota:** **No todos los prompts necesitan cada elemento de la siguiente estructura compleja**. Te animamos a experimentar e incluir o excluir elementos y ver como afecta la respuesta de Claude. Generalmente es **mejor usar muchos elementos de prompt para que tu prompt funcione primero, y luego refinar y reducir tu prompt despues**.

### Ejemplo - Chatbot de Orientacion Profesional

La siguiente estructura combina multiples elementos de ingenieria de prompts y es un buen punto de partida para prompts complejos. **El orden importa para algunos elementos**, pero no para otros. Indicaremos cuando las mejores practicas sugieren que el orden importa, pero en general, **si te ciñes a este orden, sera un buen comienzo para un prompt estelar**.

Para el siguiente ejemplo, construiremos un prompt para un juego de roles controlado donde Claude asume un rol situacional con una tarea especifica. Nuestro objetivo es hacer que Claude actue como un orientador profesional amigable.

Lee y luego ejecuta la celda de abajo para compilar los diversos elementos del prompt en un prompt completo.

In [ ]:
######################################## VARIABLES DE ENTRADA ########################################

# Primera variable de entrada - el historial de conversacion (tambien se puede agregar como mensajes previos de `user` y `assistant` en la llamada API)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Segunda variable de entrada - la pregunta del usuario
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## ELEMENTOS DEL PROMPT ########################################

##### Elemento de prompt 1: rol `user`
# Asegurate de que tu llamada a la API de Messages siempre comience con un rol `user` en el arreglo de mensajes.
# La funcion get_completion() definida arriba hara esto automaticamente por ti.

##### Elemento de prompt 2: Contexto de la tarea
# Dale a Claude contexto sobre el rol que debe asumir o que objetivos y tareas generales quieres que realice con el prompt.
# Es mejor poner el contexto al principio del cuerpo del prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Elemento de prompt 3: Contexto de tono
# Si es importante para la interaccion, dile a Claude que tono debe usar.
# Este elemento puede no ser necesario dependiendo de la tarea.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Elemento de prompt 4: Descripcion detallada de la tarea y reglas
# Expande las tareas especificas que quieres que Claude haga, asi como cualquier regla que Claude deba seguir.
# Aqui tambien puedes darle a Claude una "salida" si no tiene una respuesta o no sabe.
# Es ideal mostrar esta descripcion y reglas a un amigo para asegurarte de que este organizado logicamente y que cualquier palabra ambigua este claramente definida.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Elemento de prompt 5: Ejemplos
# Proporciona a Claude al menos un ejemplo de una respuesta ideal que pueda emular. Encierra esto en etiquetas XML <example></example>. Puedes proporcionar multiples ejemplos.
# Si proporcionas multiples ejemplos, dale a Claude contexto sobre de que es ejemplo cada uno, y encierra cada ejemplo en su propio conjunto de etiquetas XML.
# Los ejemplos son probablemente la herramienta individual mas efectiva en trabajo de conocimiento para lograr que Claude se comporte como se desea.
# Asegurate de dar a Claude ejemplos de casos limite comunes. Si tu prompt usa un scratchpad, es efectivo dar ejemplos de como deberia verse el scratchpad.
# Generalmente mas ejemplos = mejor.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Elemento de prompt 6: Datos de entrada a procesar
# Si hay datos que Claude necesita procesar dentro del prompt, incluyelos aqui dentro de etiquetas XML relevantes.
# Puedes incluir multiples datos, pero asegurate de encerrar cada uno en su propio conjunto de etiquetas XML.
# Este elemento puede no ser necesario dependiendo de la tarea. El orden tambien es flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Elemento de prompt 7: Descripcion inmediata de la tarea o solicitud #####
# "Recuerdale" a Claude o dile exactamente lo que se espera que haga inmediatamente para cumplir la tarea del prompt.
# Aqui tambien es donde pondrias variables adicionales como la pregunta del usuario.
# Generalmente no hace dano reiterar a Claude su tarea inmediata. Es mejor hacer esto hacia el final de un prompt largo.
# Esto producira mejores resultados que ponerlo al principio.
# Tambien es generalmente una buena practica poner la consulta del usuario cerca del final del prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Elemento de prompt 8: Precognicion (pensar paso a paso)
# Para tareas con multiples pasos, es bueno decirle a Claude que piense paso a paso antes de dar una respuesta
# A veces, puede que incluso tengas que decir "Antes de dar tu respuesta..." solo para asegurarte de que Claude haga esto primero.
# No es necesario con todos los prompts, aunque si se incluye, es mejor hacerlo hacia el final de un prompt largo y justo despues de la solicitud o descripcion final de la tarea inmediata.
PRECOGNITION = "Think about your answer first before you respond."

##### Elemento de prompt 9: Formato de salida
# Si hay una forma especifica en la que quieres que se formatee la respuesta de Claude, dile claramente a Claude cual es ese formato.
# Este elemento puede no ser necesario dependiendo de la tarea.
# Si lo incluyes, ponerlo hacia el final del prompt es mejor que al principio.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Elemento de prompt 10: Pre-llenado de la respuesta de Claude (si aplica)
# Un espacio para comenzar la respuesta de Claude con algunas palabras pre-llenadas para guiar el comportamiento o respuesta de Claude.
# Si quieres pre-llenar la respuesta de Claude, debes poner esto en el rol `assistant` en la llamada API.
# Este elemento puede no ser necesario dependiendo de la tarea.
PREFILL = "[Joe] <response>"



######################################## COMBINAR ELEMENTOS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Imprimir prompt completo
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

Ahora ejecutemos el prompt! Ejecuta la celda de abajo para ver la salida de Claude.

In [ ]:
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

### Ejemplo - Servicios Legales

**Los prompts dentro de la profesion legal pueden ser bastante complejos** debido a la necesidad de:
- Analizar documentos largos
- Tratar con temas complejos
- Formatear la salida de maneras muy especificas
- Seguir procesos analiticos de multiples pasos

Veamos como podemos usar la plantilla de prompt complejo para estructurar un prompt para un caso de uso legal especifico. A continuacion, hemos detallado un ejemplo de prompt para un caso de uso legal en el que pedimos a Claude que responda preguntas sobre un asunto legal usando informacion de un documento legal.

Hemos **cambiado el orden de algunos elementos** para mostrar que la estructura del prompt puede ser flexible!

**La ingenieria de prompts se trata de prueba y error cientifico**. Te animamos a mezclar y combinar, mover cosas (los elementos donde el orden no importa), y ver que funciona mejor para ti y tus necesidades.

In [ ]:
######################################## VARIABLES DE ENTRADA ########################################

# Primera variable de entrada - el documento legal
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Segunda variable de entrada - la pregunta del usuario
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## ELEMENTOS DEL PROMPT ########################################

##### Elemento de prompt 1: rol `user`
# Asegurate de que tu llamada a la API de Messages siempre comience con un rol `user` en el arreglo de mensajes.
# La funcion get_completion() definida arriba hara esto automaticamente por ti.

##### Elemento de prompt 2: Contexto de la tarea
# Dale a Claude contexto sobre el rol que debe asumir o que objetivos y tareas generales quieres que realice con el prompt.
# Es mejor poner el contexto al principio del cuerpo del prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Elemento de prompt 3: Contexto de tono
# Si es importante para la interaccion, dile a Claude que tono debe usar.
# Este elemento puede no ser necesario dependiendo de la tarea.
TONE_CONTEXT = ""

##### Elemento de prompt 4: Datos de entrada a procesar
# Si hay datos que Claude necesita procesar dentro del prompt, incluyelos aqui dentro de etiquetas XML relevantes.
# Puedes incluir multiples datos, pero asegurate de encerrar cada uno en su propio conjunto de etiquetas XML.
# Este elemento puede no ser necesario dependiendo de la tarea. El orden tambien es flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Elemento de prompt 5: Ejemplos
# Proporciona a Claude al menos un ejemplo de una respuesta ideal que pueda emular. Encierra esto en etiquetas XML <example></example>. Puedes proporcionar multiples ejemplos.
# Si proporcionas multiples ejemplos, dale a Claude contexto sobre de que es ejemplo cada uno, y encierra cada ejemplo en su propio conjunto de etiquetas XML.
# Los ejemplos son probablemente la herramienta individual mas efectiva en trabajo de conocimiento para lograr que Claude se comporte como se desea.
# Asegurate de dar a Claude ejemplos de casos limite comunes. Si tu prompt usa un scratchpad, es efectivo dar ejemplos de como deberia verse el scratchpad.
# Generalmente mas ejemplos = mejor.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Elemento de prompt 6: Descripcion detallada de la tarea y reglas
# Expande las tareas especificas que quieres que Claude haga, asi como cualquier regla que Claude deba seguir.
# Aqui tambien puedes darle a Claude una "salida" si no tiene una respuesta o no sabe.
# Es ideal mostrar esta descripcion y reglas a un amigo para asegurarte de que este organizado logicamente y que cualquier palabra ambigua este claramente definida.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Elemento de prompt 7: Descripcion inmediata de la tarea o solicitud #####
# "Recuerdale" a Claude o dile exactamente lo que se espera que haga inmediatamente para cumplir la tarea del prompt.
# Aqui tambien es donde pondrias variables adicionales como la pregunta del usuario.
# Generalmente no hace dano reiterar a Claude su tarea inmediata. Es mejor hacer esto hacia el final de un prompt largo.
# Esto producira mejores resultados que ponerlo al principio.
# Tambien es generalmente una buena practica poner la consulta del usuario cerca del final del prompt.
IMMEDIATE_TASK = ""

##### Elemento de prompt 8: Precognicion (pensar paso a paso)
# Para tareas con multiples pasos, es bueno decirle a Claude que piense paso a paso antes de dar una respuesta
# A veces, puede que incluso tengas que decir "Antes de dar tu respuesta..." solo para asegurarte de que Claude haga esto primero.
# No es necesario con todos los prompts, aunque si se incluye, es mejor hacerlo hacia el final de un prompt largo y justo despues de la solicitud o descripcion final de la tarea inmediata.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Elemento de prompt 9: Formato de salida
# Si hay una forma especifica en la que quieres que se formatee la respuesta de Claude, dile claramente a Claude cual es ese formato.
# Este elemento puede no ser necesario dependiendo de la tarea.
# Si lo incluyes, ponerlo hacia el final del prompt es mejor que al principio.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Elemento de prompt 10: Pre-llenado de la respuesta de Claude (si aplica)
# Un espacio para comenzar la respuesta de Claude con algunas palabras pre-llenadas para guiar el comportamiento o respuesta de Claude.
# Si quieres pre-llenar la respuesta de Claude, debes poner esto en el rol `assistant` en la llamada API.
# Este elemento puede no ser necesario dependiendo de la tarea.
PREFILL = "<relevant_quotes>"



######################################## COMBINAR ELEMENTOS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Imprimir prompt completo
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

Ahora ejecutemos el prompt! Ejecuta la celda de abajo para ver la salida de Claude.

In [ ]:
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 9.1 - Chatbot de Servicios Financieros](#ejercicio-91---chatbot-de-servicios-financieros)
- [Ejercicio 9.2 - Codebot](#ejercicio-92---codebot)

### Ejercicio 9.1 - Chatbot de Servicios Financieros
Los prompts dentro de la profesion financiera tambien pueden ser bastante complejos por razones similares a los prompts legales. Aqui hay un ejercicio para un caso de uso financiero, donde Claude se usa para **analizar informacion fiscal y responder preguntas**. Al igual que con el ejemplo de servicios legales, hemos cambiado el orden de algunos elementos, ya que nuestro prompt solucion tiene mas sentido con un flujo diferente (sin embargo, otras estructuras tambien funcionarian).

Te sugerimos que leas el contenido de las variables (en este caso, `{QUESTION}` y `{TAX_CODE}`) para entender con que contenido se espera que Claude trabaje. Asegurate de referenciar `{QUESTION}` y `{TAX_CODE}` directamente en tu prompt en algun lugar (usando sintaxis f-string como en los otros ejemplos) para que el contenido real de la variable pueda ser sustituido.

Completa los campos de elementos del prompt con contenido que coincida con la descripcion y los ejemplos que has visto en los ejemplos anteriores de prompts complejos. Una vez que hayas completado todos los elementos del prompt que desees, ejecuta la celda para ver el prompt concatenado asi como la respuesta de Claude.

Recuerda que la ingenieria de prompts raramente es puramente formulaica, especialmente para prompts grandes y complejos! Es importante desarrollar casos de prueba y **probar una variedad de prompts y estructuras de prompts para ver que funciona mejor para cada situacion**. Ten en cuenta que si *cambias* el orden de los elementos del prompt, tambien debes recordar cambiar el orden de la concatenacion en la seccion `COMBINAR ELEMENTOS`.

In [ ]:
######################################## VARIABLES DE ENTRADA ########################################

# Primera variable de entrada - la pregunta del usuario
QUESTION = "How long do I have to make an 83b election?"

# Segunda variable de entrada - el documento del codigo fiscal que Claude usara para responder la pregunta del usuario
TAX_CODE = """
(a)General rule
If, in connection with the performance of services, property is transferred to any person other than the person for whom such services are performed, the excess of—
(1)the fair market value of such property (determined without regard to any restriction other than a restriction which by its terms will never lapse) at the first time the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, over
(2)the amount (if any) paid for such property,
shall be included in the gross income of the person who performed such services in the first taxable year in which the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever is applicable. The preceding sentence shall not apply if such person sells or otherwise disposes of such property in an arm’s length transaction before his rights in such property become transferable or not subject to a substantial risk of forfeiture.
(b)Election to include in gross income in year of transfer
(1)In general
Any person who performs services in connection with which property is transferred to any person may elect to include in his gross income for the taxable year in which such property is transferred, the excess of—
(A)the fair market value of such property at the time of transfer (determined without regard to any restriction other than a restriction which by its terms will never lapse), over
(B)the amount (if any) paid for such property.
If such election is made, subsection (a) shall not apply with respect to the transfer of such property, and if such property is subsequently forfeited, no deduction shall be allowed in respect of such forfeiture.
(2)Election
An election under paragraph (1) with respect to any transfer of property shall be made in such manner as the Secretary prescribes and shall be made not later than 30 days after the date of such transfer. Such election may not be revoked except with the consent of the Secretary.

(c)Special rules
For purposes of this section—
(1)Substantial risk of forfeiture
The rights of a person in property are subject to a substantial risk of forfeiture if such person’s rights to full enjoyment of such property are conditioned upon the future performance of substantial services by any individual.

(2)Transferability of property
The rights of a person in property are transferable only if the rights in such property of any transferee are not subject to a substantial risk of forfeiture.

(3)Sales which may give rise to suit under section 16(b) of the Securities Exchange Act of 1934
So long as the sale of property at a profit could subject a person to suit under section 16(b) of the Securities Exchange Act of 1934, such person’s rights in such property are—
(A)subject to a substantial risk of forfeiture, and
(B)not transferable.
(4)For purposes of determining an individual’s basis in property transferred in connection with the performance of services, rules similar to the rules of section 72(w) shall apply.
(d)Certain restrictions which will never lapse
(1)Valuation
In the case of property subject to a restriction which by its terms will never lapse, and which allows the transferee to sell such property only at a price determined under a formula, the price so determined shall be deemed to be the fair market value of the property unless established to the contrary by the Secretary, and the burden of proof shall be on the Secretary with respect to such value.

(2)Cancellation
If, in the case of property subject to a restriction which by its terms will never lapse, the restriction is canceled, then, unless the taxpayer establishes—
(A)that such cancellation was not compensatory, and
(B)that the person, if any, who would be allowed a deduction if the cancellation were treated as compensatory, will treat the transaction as not compensatory, as evidenced in such manner as the Secretary shall prescribe by regulations,
the excess of the fair market value of the property (computed without regard to the restrictions) at the time of cancellation over the sum of—
(C)the fair market value of such property (computed by taking the restriction into account) immediately before the cancellation, and
(D)the amount, if any, paid for the cancellation,
shall be treated as compensation for the taxable year in which such cancellation occurs.
(e)Applicability of section
This section shall not apply to—
(1)a transaction to which section 421 applies,
(2)a transfer to or from a trust described in section 401(a) or a transfer under an annuity plan which meets the requirements of section 404(a)(2),
(3)the transfer of an option without a readily ascertainable fair market value,
(4)the transfer of property pursuant to the exercise of an option with a readily ascertainable fair market value at the date of grant, or
(5)group-term life insurance to which section 79 applies.
(f)Holding period
In determining the period for which the taxpayer has held property to which subsection (a) applies, there shall be included only the period beginning at the first time his rights in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier.

(g)Certain exchanges
If property to which subsection (a) applies is exchanged for property subject to restrictions and conditions substantially similar to those to which the property given in such exchange was subject, and if section 354, 355, 356, or 1036 (or so much of section 1031 as relates to section 1036) applied to such exchange, or if such exchange was pursuant to the exercise of a conversion privilege—
(1)such exchange shall be disregarded for purposes of subsection (a), and
(2)the property received shall be treated as property to which subsection (a) applies.
(h)Deduction by employer
In the case of a transfer of property to which this section applies or a cancellation of a restriction described in subsection (d), there shall be allowed as a deduction under section 162, to the person for whom were performed the services in connection with which such property was transferred, an amount equal to the amount included under subsection (a), (b), or (d)(2) in the gross income of the person who performed such services. Such deduction shall be allowed for the taxable year of such person in which or with which ends the taxable year in which such amount is included in the gross income of the person who performed such services.

(i)Qualified equity grants
(1)In general
For purposes of this subtitle—
(A)Timing of inclusion
If qualified stock is transferred to a qualified employee who makes an election with respect to such stock under this subsection, subsection (a) shall be applied by including the amount determined under such subsection with respect to such stock in income of the employee in the taxable year determined under subparagraph (B) in lieu of the taxable year described in subsection (a).

(B)Taxable year determined
The taxable year determined under this subparagraph is the taxable year of the employee which includes the earliest of—
(i)the first date such qualified stock becomes transferable (including, solely for purposes of this clause, becoming transferable to the employer),
(ii)the date the employee first becomes an excluded employee,
(iii)the first date on which any stock of the corporation which issued the qualified stock becomes readily tradable on an established securities market (as determined by the Secretary, but not including any market unless such market is recognized as an established securities market by the Secretary for purposes of a provision of this title other than this subsection),
(iv)the date that is 5 years after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, or
(v)the date on which the employee revokes (at such time and in such manner as the Secretary provides) the election under this subsection with respect to such stock.
(2)Qualified stock
(A)In general
For purposes of this subsection, the term “qualified stock” means, with respect to any qualified employee, any stock in a corporation which is the employer of such employee, if—
(i)such stock is received—
(I)in connection with the exercise of an option, or
(II)in settlement of a restricted stock unit, and
(ii)such option or restricted stock unit was granted by the corporation—
(I)in connection with the performance of services as an employee, and
(II)during a calendar year in which such corporation was an eligible corporation.
(B)Limitation
The term “qualified stock” shall not include any stock if the employee may sell such stock to, or otherwise receive cash in lieu of stock from, the corporation at the time that the rights of the employee in such stock first become transferable or not subject to a substantial risk of forfeiture.

(C)Eligible corporation
For purposes of subparagraph (A)(ii)(II)—
(i)In general
The term “eligible corporation” means, with respect to any calendar year, any corporation if—
(I)no stock of such corporation (or any predecessor of such corporation) is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) during any preceding calendar year, and
(II)such corporation has a written plan under which, in such calendar year, not less than 80 percent of all employees who provide services to such corporation in the United States (or any possession of the United States) are granted stock options, or are granted restricted stock units, with the same rights and privileges to receive qualified stock.
(ii)Same rights and privileges
For purposes of clause (i)(II)—
(I)except as provided in subclauses (II) and (III), the determination of rights and privileges with respect to stock shall be made in a similar manner as under section 423(b)(5),
(II)employees shall not fail to be treated as having the same rights and privileges to receive qualified stock solely because the number of shares available to all employees is not equal in amount, so long as the number of shares available to each employee is more than a de minimis amount, and
(III)rights and privileges with respect to the exercise of an option shall not be treated as the same as rights and privileges with respect to the settlement of a restricted stock unit.
(iii)Employee
For purposes of clause (i)(II), the term “employee” shall not include any employee described in section 4980E(d)(4) or any excluded employee.

(iv)Special rule for calendar years before 2018
In the case of any calendar year beginning before January 1, 2018, clause (i)(II) shall be applied without regard to whether the rights and privileges with respect to the qualified stock are the same.

(3)Qualified employee; excluded employee
For purposes of this subsection—
(A)In general
The term “qualified employee” means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.
(B)Excluded employee
The term “excluded employee” means, with respect to any corporation, any individual—
(i)who is a 1-percent owner (within the meaning of section 416(i)(1)(B)(ii)) at any time during the calendar year or who was such a 1 percent owner at any time during the 10 preceding calendar years,
(ii)who is or has been at any prior time—
(I)the chief executive officer of such corporation or an individual acting in such a capacity, or
(II)the chief financial officer of such corporation or an individual acting in such a capacity,
(iii)who bears a relationship described in section 318(a)(1) to any individual described in subclause (I) or (II) of clause (ii), or
(iv)who is one of the 4 highest compensated officers of such corporation for the taxable year, or was one of the 4 highest compensated officers of such corporation for any of the 10 preceding taxable years, determined with respect to each such taxable year on the basis of the shareholder disclosure rules for compensation under the Securities Exchange Act of 1934 (as if such rules applied to such corporation).
(4)Election
(A)Time for making election
An election with respect to qualified stock shall be made under this subsection no later than 30 days after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, and shall be made in a manner similar to the manner in which an election is made under subsection (b).

(B)Limitations
No election may be made under this section with respect to any qualified stock if—
(i)the qualified employee has made an election under subsection (b) with respect to such qualified stock,
(ii)any stock of the corporation which issued the qualified stock is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) at any time before the election is made, or
(iii)such corporation purchased any of its outstanding stock in the calendar year preceding the calendar year which includes the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, unless—
(I)not less than 25 percent of the total dollar amount of the stock so purchased is deferral stock, and
(II)the determination of which individuals from whom deferral stock is purchased is made on a reasonable basis.
(C)Definitions and special rules related to limitation on stock redemptions
(i)Deferral stock
For purposes of this paragraph, the term “deferral stock” means stock with respect to which an election is in effect under this subsection.

(ii)Deferral stock with respect to any individual not taken into account if individual holds deferral stock with longer deferral period
Stock purchased by a corporation from any individual shall not be treated as deferral stock for purposes of subparagraph (B)(iii) if such individual (immediately after such purchase) holds any deferral stock with respect to which an election has been in effect under this subsection for a longer period than the election with respect to the stock so purchased.

(iii)Purchase of all outstanding deferral stock
The requirements of subclauses (I) and (II) of subparagraph (B)(iii) shall be treated as met if the stock so purchased includes all of the corporation’s outstanding deferral stock.

(iv)Reporting
Any corporation which has outstanding deferral stock as of the beginning of any calendar year and which purchases any of its outstanding stock during such calendar year shall include on its return of tax for the taxable year in which, or with which, such calendar year ends the total dollar amount of its outstanding stock so purchased during such calendar year and such other information as the Secretary requires for purposes of administering this paragraph.

(5)Controlled groups
For purposes of this subsection, all persons treated as a single employer under section 414(b) shall be treated as 1 corporation.

(6)Notice requirement
Any corporation which transfers qualified stock to a qualified employee shall, at the time that (or a reasonable period before) an amount attributable to such stock would (but for this subsection) first be includible in the gross income of such employee—
(A)certify to such employee that such stock is qualified stock, and
(B)notify such employee—
(i)that the employee may be eligible to elect to defer income on such stock under this subsection, and
(ii)that, if the employee makes such an election—
(I)the amount of income recognized at the end of the deferral period will be based on the value of the stock at the time at which the rights of the employee in such stock first become transferable or not subject to substantial risk of forfeiture, notwithstanding whether the value of the stock has declined during the deferral period,
(II)the amount of such income recognized at the end of the deferral period will be subject to withholding under section 3401(i) at the rate determined under section 3402(t), and
(III)the responsibilities of the employee (as determined by the Secretary under paragraph (3)(A)(ii)) with respect to such withholding.
(7)Restricted stock units
This section (other than this subsection), including any election under subsection (b), shall not apply to restricted stock units.
"""



######################################## ELEMENTOS DEL PROMPT ########################################

##### Elemento de prompt 1: rol `user`
# Asegurate de que tu llamada a la API de Messages siempre comience con un rol `user` en el arreglo de mensajes.
# La funcion get_completion() definida arriba hara esto automaticamente por ti.

##### Elemento de prompt 2: Contexto de la tarea
# Dale a Claude contexto sobre el rol que debe asumir o que objetivos y tareas generales quieres que realice con el prompt.
# Es mejor poner el contexto al principio del cuerpo del prompt.
TASK_CONTEXT = ""

##### Elemento de prompt 3: Contexto de tono
# Si es importante para la interaccion, dile a Claude que tono debe usar.
# Este elemento puede no ser necesario dependiendo de la tarea.
TONE_CONTEXT = ""

##### Elemento de prompt 4: Datos de entrada a procesar
# Si hay datos que Claude necesita procesar dentro del prompt, incluyelos aqui dentro de etiquetas XML relevantes.
# Puedes incluir multiples datos, pero asegurate de encerrar cada uno en su propio conjunto de etiquetas XML.
# Este elemento puede no ser necesario dependiendo de la tarea. El orden tambien es flexible.
INPUT_DATA = ""

##### Elemento de prompt 5: Ejemplos
# Proporciona a Claude al menos un ejemplo de una respuesta ideal que pueda emular. Encierra esto en etiquetas XML <example></example>. Puedes proporcionar multiples ejemplos.
# Si proporcionas multiples ejemplos, dale a Claude contexto sobre de que es ejemplo cada uno, y encierra cada ejemplo en su propio conjunto de etiquetas XML.
# Los ejemplos son probablemente la herramienta individual mas efectiva en trabajo de conocimiento para lograr que Claude se comporte como se desea.
# Asegurate de dar a Claude ejemplos de casos limite comunes. Si tu prompt usa un scratchpad, es efectivo dar ejemplos de como deberia verse el scratchpad.
# Generalmente mas ejemplos = mejor.
EXAMPLES = ""

##### Elemento de prompt 6: Descripcion detallada de la tarea y reglas
# Expande las tareas especificas que quieres que Claude haga, asi como cualquier regla que Claude deba seguir.
# Aqui tambien puedes darle a Claude una "salida" si no tiene una respuesta o no sabe.
# Es ideal mostrar esta descripcion y reglas a un amigo para asegurarte de que este organizado logicamente y que cualquier palabra ambigua este claramente definida.
TASK_DESCRIPTION = ""

##### Elemento de prompt 7: Descripcion inmediata de la tarea o solicitud #####
# "Recuerdale" a Claude o dile exactamente lo que se espera que haga inmediatamente para cumplir la tarea del prompt.
# Aqui tambien es donde pondrias variables adicionales como la pregunta del usuario.
# Generalmente no hace dano reiterar a Claude su tarea inmediata. Es mejor hacer esto hacia el final de un prompt largo.
# Esto producira mejores resultados que ponerlo al principio.
# Tambien es generalmente una buena practica poner la consulta del usuario cerca del final del prompt.
IMMEDIATE_TASK = ""

##### Elemento de prompt 8: Precognicion (pensar paso a paso)
# Para tareas con multiples pasos, es bueno decirle a Claude que piense paso a paso antes de dar una respuesta
# A veces, puede que incluso tengas que decir "Antes de dar tu respuesta..." solo para asegurarte de que Claude haga esto primero.
# No es necesario con todos los prompts, aunque si se incluye, es mejor hacerlo hacia el final de un prompt largo y justo despues de la solicitud o descripcion final de la tarea inmediata.
PRECOGNITION = ""

##### Elemento de prompt 9: Formato de salida
# Si hay una forma especifica en la que quieres que se formatee la respuesta de Claude, dile claramente a Claude cual es ese formato.
# Este elemento puede no ser necesario dependiendo de la tarea.
# Si lo incluyes, ponerlo hacia el final del prompt es mejor que al principio.
OUTPUT_FORMATTING = ""

##### Elemento de prompt 10: Pre-llenado de la respuesta de Claude (si aplica)
# Un espacio para comenzar la respuesta de Claude con algunas palabras pre-llenadas para guiar el comportamiento o respuesta de Claude.
# Si quieres pre-llenar la respuesta de Claude, debes poner esto en el rol `assistant` en la llamada API.
# Este elemento puede no ser necesario dependiendo de la tarea.
PREFILL = ""



######################################## COMBINAR ELEMENTOS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Imprimir prompt completo
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Si quieres ver una posible solucion, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_9_1_solution; print(exercise_9_1_solution)

### Ejercicio 9.2 - Codebot
En este ejercicio, escribiremos un prompt para un **bot de asistencia y ensenanza de codigo que lee codigo y ofrece correcciones guiadas cuando sea apropiado**. Completa los campos de elementos del prompt con contenido que coincida con la descripcion y los ejemplos que has visto en los ejemplos anteriores de prompts complejos. Una vez que hayas completado todos los elementos del prompt que desees, ejecuta la celda para ver el prompt concatenado asi como la respuesta de Claude.

Te sugerimos que leas el contenido de las variables (en este caso, `{CODE}`) para entender con que contenido se espera que Claude trabaje. Asegurate de referenciar `{CODE}` directamente en tu prompt en algun lugar (usando sintaxis f-string como en los otros ejemplos) para que el contenido real de la variable pueda ser sustituido.

In [ ]:
######################################## VARIABLES DE ENTRADA ########################################

# Variable de entrada - el codigo que Claude necesita leer y ayudar al usuario a corregir
CODE = """
# Funcion para imprimir inversos multiplicativos
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## ELEMENTOS DEL PROMPT ########################################

##### Elemento de prompt 1: rol `user`
# Asegurate de que tu llamada a la API de Messages siempre comience con un rol `user` en el arreglo de mensajes.
# La funcion get_completion() definida arriba hara esto automaticamente por ti.

##### Elemento de prompt 2: Contexto de la tarea
# Dale a Claude contexto sobre el rol que debe asumir o que objetivos y tareas generales quieres que realice con el prompt.
# Es mejor poner el contexto al principio del cuerpo del prompt.
TASK_CONTEXT = ""

##### Elemento de prompt 3: Contexto de tono
# Si es importante para la interaccion, dile a Claude que tono debe usar.
# Este elemento puede no ser necesario dependiendo de la tarea.
TONE_CONTEXT = ""

##### Elemento de prompt 4: Descripcion detallada de la tarea y reglas
# Expande las tareas especificas que quieres que Claude haga, asi como cualquier regla que Claude deba seguir.
# Aqui tambien puedes darle a Claude una "salida" si no tiene una respuesta o no sabe.
# Es ideal mostrar esta descripcion y reglas a un amigo para asegurarte de que este organizado logicamente y que cualquier palabra ambigua este claramente definida.
TASK_DESCRIPTION = ""

##### Elemento de prompt 5: Ejemplos
# Proporciona a Claude al menos un ejemplo de una respuesta ideal que pueda emular. Encierra esto en etiquetas XML <example></example>. Puedes proporcionar multiples ejemplos.
# Si proporcionas multiples ejemplos, dale a Claude contexto sobre de que es ejemplo cada uno, y encierra cada ejemplo en su propio conjunto de etiquetas XML.
# Los ejemplos son probablemente la herramienta individual mas efectiva en trabajo de conocimiento para lograr que Claude se comporte como se desea.
# Asegurate de dar a Claude ejemplos de casos limite comunes. Si tu prompt usa un scratchpad, es efectivo dar ejemplos de como deberia verse el scratchpad.
# Generalmente mas ejemplos = mejor.
EXAMPLES = ""

##### Elemento de prompt 6: Datos de entrada a procesar
# Si hay datos que Claude necesita procesar dentro del prompt, incluyelos aqui dentro de etiquetas XML relevantes.
# Puedes incluir multiples datos, pero asegurate de encerrar cada uno en su propio conjunto de etiquetas XML.
# Este elemento puede no ser necesario dependiendo de la tarea. El orden tambien es flexible.
INPUT_DATA = ""

##### Elemento de prompt 7: Descripcion inmediata de la tarea o solicitud #####
# "Recuerdale" a Claude o dile exactamente lo que se espera que haga inmediatamente para cumplir la tarea del prompt.
# Aqui tambien es donde pondrias variables adicionales como la pregunta del usuario.
# Generalmente no hace dano reiterar a Claude su tarea inmediata. Es mejor hacer esto hacia el final de un prompt largo.
# Esto producira mejores resultados que ponerlo al principio.
# Tambien es generalmente una buena practica poner la consulta del usuario cerca del final del prompt.
IMMEDIATE_TASK = ""

##### Elemento de prompt 8: Precognicion (pensar paso a paso)
# Para tareas con multiples pasos, es bueno decirle a Claude que piense paso a paso antes de dar una respuesta
# A veces, puede que incluso tengas que decir "Antes de dar tu respuesta..." solo para asegurarte de que Claude haga esto primero.
# No es necesario con todos los prompts, aunque si se incluye, es mejor hacerlo hacia el final de un prompt largo y justo despues de la solicitud o descripcion final de la tarea inmediata.
PRECOGNITION = ""

##### Elemento de prompt 9: Formato de salida
# Si hay una forma especifica en la que quieres que se formatee la respuesta de Claude, dile claramente a Claude cual es ese formato.
# Este elemento puede no ser necesario dependiendo de la tarea.
# Si lo incluyes, ponerlo hacia el final del prompt es mejor que al principio.
OUTPUT_FORMATTING = ""

##### Elemento de prompt 10: Pre-llenado de la respuesta de Claude (si aplica)
# Un espacio para comenzar la respuesta de Claude con algunas palabras pre-llenadas para guiar el comportamiento o respuesta de Claude.
# Si quieres pre-llenar la respuesta de Claude, debes poner esto en el rol `assistant` en la llamada API.
# Este elemento puede no ser necesario dependiendo de la tarea.
PREFILL = ""



######################################## COMBINAR ELEMENTOS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Imprimir prompt completo
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Si quieres ver una posible solucion, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_9_2_solution; print(exercise_9_2_solution)

---

## Felicidades y Proximos Pasos!

Si completaste todos los ejercicios, **ahora estas en el 0.1% superior de los susurradores de LLM**. Uno de la elite!

Las tecnicas que has aprendido, desde pensar paso a paso hasta asignar roles, usar ejemplos y la escritura clara en general, pueden ser **combinadas, remezcladas y adaptadas de innumerables maneras**.

La ingenieria de prompts es una disciplina muy nueva, asi que manten una mente abierta. Tu podrias ser quien descubra el proximo gran truco de prompting.

Si quieres ver **mas ejemplos de buenos prompts** para inspiracion:
- Aprende de ejemplos de prompts listos para produccion en nuestro [cookbook](https://anthropic.com/cookbook)
- Lee nuestra [guia de prompting](https://docs.anthropic.com/claude/docs/prompt-engineering)
- Consulta nuestra [biblioteca de prompts](https://anthropic.com/prompts) para inspiracion
- Prueba nuestro [metaprompt](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental) experimental para que Claude escriba plantillas de prompts por ti!
- Haz preguntas en nuestro [servidor de Discord](https://anthropic.com/discord)
- Aprende sobre los [parametros de la API de Anthropic](https://docs.anthropic.com/claude/reference/complete_post) como temperature y `max_tokens`
- Si te sientes academico, lee algunos [articulos](https://www.promptingguide.ai/papers) sobre ingenieria de prompts
- Practica construyendo prompts para que Claude haga algo que te interese

Si quieres aprender sobre algunas tecnicas de prompting verdaderamente avanzadas mas alla del alcance de este tutorial, continua al apendice! Pero primero, ejecuta la celda de abajo.

In [ ]:
# Prompt
PROMPT = "Write an ode to a fabulous student who has just completed a course on prompt engineering, in the form of a sonnet."

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompts mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
######################################## VARIABLES DE ENTRADA ########################################

# Primera variable de entrada - el historial de conversacion (tambien se puede agregar como mensajes previos de `user` y `assistant` en la llamada API)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Segunda variable de entrada - la pregunta del usuario
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## ELEMENTOS DEL PROMPT ########################################

##### Elemento de prompt 1: rol `user`
# Asegurate de que tu llamada a la API de Messages siempre comience con un rol `user` en el arreglo de mensajes.
# La funcion get_completion() definida arriba hara esto automaticamente por ti.

##### Elemento de prompt 2: Contexto de la tarea
# Dale a Claude contexto sobre el rol que debe asumir o que objetivos y tareas generales quieres que realice con el prompt.
# Es mejor poner el contexto al principio del cuerpo del prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Elemento de prompt 3: Contexto de tono
# Si es importante para la interaccion, dile a Claude que tono debe usar.
# Este elemento puede no ser necesario dependiendo de la tarea.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Elemento de prompt 4: Descripcion detallada de la tarea y reglas
# Expande las tareas especificas que quieres que Claude haga, asi como cualquier regla que Claude deba seguir.
# Aqui tambien puedes darle a Claude una "salida" si no tiene una respuesta o no sabe.
# Es ideal mostrar esta descripcion y reglas a un amigo para asegurarte de que este organizado logicamente y que cualquier palabra ambigua este claramente definida.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Elemento de prompt 5: Ejemplos
# Proporciona a Claude al menos un ejemplo de una respuesta ideal que pueda emular. Encierra esto en etiquetas XML <example></example>. Puedes proporcionar multiples ejemplos.
# Si proporcionas multiples ejemplos, dale a Claude contexto sobre de que es ejemplo cada uno, y encierra cada ejemplo en su propio conjunto de etiquetas XML.
# Los ejemplos son probablemente la herramienta individual mas efectiva en trabajo de conocimiento para lograr que Claude se comporte como se desea.
# Asegurate de dar a Claude ejemplos de casos limite comunes. Si tu prompt usa un scratchpad, es efectivo dar ejemplos de como deberia verse el scratchpad.
# Generalmente mas ejemplos = mejor.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Elemento de prompt 6: Datos de entrada a procesar
# Si hay datos que Claude necesita procesar dentro del prompt, incluyelos aqui dentro de etiquetas XML relevantes.
# Puedes incluir multiples datos, pero asegurate de encerrar cada uno en su propio conjunto de etiquetas XML.
# Este elemento puede no ser necesario dependiendo de la tarea. El orden tambien es flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Elemento de prompt 7: Descripcion inmediata de la tarea o solicitud #####
# "Recuerdale" a Claude o dile exactamente lo que se espera que haga inmediatamente para cumplir la tarea del prompt.
# Aqui tambien es donde pondrias variables adicionales como la pregunta del usuario.
# Generalmente no hace dano reiterar a Claude su tarea inmediata. Es mejor hacer esto hacia el final de un prompt largo.
# Esto producira mejores resultados que ponerlo al principio.
# Tambien es generalmente una buena practica poner la consulta del usuario cerca del final del prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Elemento de prompt 8: Precognicion (pensar paso a paso)
# Para tareas con multiples pasos, es bueno decirle a Claude que piense paso a paso antes de dar una respuesta
# A veces, puede que incluso tengas que decir "Antes de dar tu respuesta..." solo para asegurarte de que Claude haga esto primero.
# No es necesario con todos los prompts, aunque si se incluye, es mejor hacerlo hacia el final de un prompt largo y justo despues de la solicitud o descripcion final de la tarea inmediata.
PRECOGNITION = "Think about your answer first before you respond."

##### Elemento de prompt 9: Formato de salida
# Si hay una forma especifica en la que quieres que se formatee la respuesta de Claude, dile claramente a Claude cual es ese formato.
# Este elemento puede no ser necesario dependiendo de la tarea.
# Si lo incluyes, ponerlo hacia el final del prompt es mejor que al principio.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Elemento de prompt 10: Pre-llenado de la respuesta de Claude (si aplica)
# Un espacio para comenzar la respuesta de Claude con algunas palabras pre-llenadas para guiar el comportamiento o respuesta de Claude.
# Si quieres pre-llenar la respuesta de Claude, debes poner esto en el rol `assistant` en la llamada API.
# Este elemento puede no ser necesario dependiendo de la tarea.
PREFILL = "[Joe] <response>"



######################################## COMBINAR ELEMENTOS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Imprimir prompt completo
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
######################################## VARIABLES DE ENTRADA ########################################

# Primera variable de entrada - el documento legal
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Segunda variable de entrada - la pregunta del usuario
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## ELEMENTOS DEL PROMPT ########################################

##### Elemento de prompt 1: rol `user`
# Asegurate de que tu llamada a la API de Messages siempre comience con un rol `user` en el arreglo de mensajes.
# La funcion get_completion() definida arriba hara esto automaticamente por ti.

##### Elemento de prompt 2: Contexto de la tarea
# Dale a Claude contexto sobre el rol que debe asumir o que objetivos y tareas generales quieres que realice con el prompt.
# Es mejor poner el contexto al principio del cuerpo del prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Elemento de prompt 3: Contexto de tono
# Si es importante para la interaccion, dile a Claude que tono debe usar.
# Este elemento puede no ser necesario dependiendo de la tarea.
TONE_CONTEXT = ""

##### Elemento de prompt 4: Datos de entrada a procesar
# Si hay datos que Claude necesita procesar dentro del prompt, incluyelos aqui dentro de etiquetas XML relevantes.
# Puedes incluir multiples datos, pero asegurate de encerrar cada uno en su propio conjunto de etiquetas XML.
# Este elemento puede no ser necesario dependiendo de la tarea. El orden tambien es flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Elemento de prompt 5: Ejemplos
# Proporciona a Claude al menos un ejemplo de una respuesta ideal que pueda emular. Encierra esto en etiquetas XML <example></example>. Puedes proporcionar multiples ejemplos.
# Si proporcionas multiples ejemplos, dale a Claude contexto sobre de que es ejemplo cada uno, y encierra cada ejemplo en su propio conjunto de etiquetas XML.
# Los ejemplos son probablemente la herramienta individual mas efectiva en trabajo de conocimiento para lograr que Claude se comporte como se desea.
# Asegurate de dar a Claude ejemplos de casos limite comunes. Si tu prompt usa un scratchpad, es efectivo dar ejemplos de como deberia verse el scratchpad.
# Generalmente mas ejemplos = mejor.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Elemento de prompt 6: Descripcion detallada de la tarea y reglas
# Expande las tareas especificas que quieres que Claude haga, asi como cualquier regla que Claude deba seguir.
# Aqui tambien puedes darle a Claude una "salida" si no tiene una respuesta o no sabe.
# Es ideal mostrar esta descripcion y reglas a un amigo para asegurarte de que este organizado logicamente y que cualquier palabra ambigua este claramente definida.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Elemento de prompt 7: Descripcion inmediata de la tarea o solicitud #####
# "Recuerdale" a Claude o dile exactamente lo que se espera que haga inmediatamente para cumplir la tarea del prompt.
# Aqui tambien es donde pondrias variables adicionales como la pregunta del usuario.
# Generalmente no hace dano reiterar a Claude su tarea inmediata. Es mejor hacer esto hacia el final de un prompt largo.
# Esto producira mejores resultados que ponerlo al principio.
# Tambien es generalmente una buena practica poner la consulta del usuario cerca del final del prompt.
IMMEDIATE_TASK = ""

##### Elemento de prompt 8: Precognicion (pensar paso a paso)
# Para tareas con multiples pasos, es bueno decirle a Claude que piense paso a paso antes de dar una respuesta
# A veces, puede que incluso tengas que decir "Antes de dar tu respuesta..." solo para asegurarte de que Claude haga esto primero.
# No es necesario con todos los prompts, aunque si se incluye, es mejor hacerlo hacia el final de un prompt largo y justo despues de la solicitud o descripcion final de la tarea inmediata.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Elemento de prompt 9: Formato de salida
# Si hay una forma especifica en la que quieres que se formatee la respuesta de Claude, dile claramente a Claude cual es ese formato.
# Este elemento puede no ser necesario dependiendo de la tarea.
# Si lo incluyes, ponerlo hacia el final del prompt es mejor que al principio.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Elemento de prompt 10: Pre-llenado de la respuesta de Claude (si aplica)
# Un espacio para comenzar la respuesta de Claude con algunas palabras pre-llenadas para guiar el comportamiento o respuesta de Claude.
# Si quieres pre-llenar la respuesta de Claude, debes poner esto en el rol `assistant` en la llamada API.
# Este elemento puede no ser necesario dependiendo de la tarea.
PREFILL = "<relevant_quotes>"



######################################## COMBINAR ELEMENTOS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Imprimir prompt completo
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))